# 네이버 지도 검색 결과 스크랩

![naver_map](https://lh3.google.com/u/0/d/16kx-lhpaCHA82IbspowO1ygQhZAuOk7C=w3360-h1878-iv1)


### 가게 정보 및 img 스크랩

[상황 설명]  
>직장인 A 씨는 웨딩업체 사진 작가입니다.  
서울의 웨딩사진전문 업체에 대한 정보와 각 업체에서 올린 사진을 모으고 싶은데요.  
네이버 지도에서 특정 지역 타겟팅 후 주변에 위치한 웨딩사진전문 업체를 검색하여  
업체에 대한 정보와 네이버 플레이스에 등록된 사진을 모으려고 합니다.

- 당연한 이야기지만 스크랩은 언제든지 막힐 수 있습니다.

In [ ]:
import numpy as np
import pandas as pd
import urllib.request
import requests
import os
import json
import time

> url은 직접 네이버 지도 접속 후 특정 지역을 타겟팅하여 검색한 결과를 가져옵니다.

In [ ]:
# 검색명은 변경 가능
url = f"https://map.naver.com/v5/api/search?caller=pcweb&query=웨딩사진전문&type=all&searchCoord=126.75292692474181;37.539870164829594&page=3&displayCount=100&isPlaceRecommendationReplace=true&lang=ko"
res = requests.get(url)
map_data = json.loads(res.text)

In [ ]:
# time.sleep 시간을 랜덤추출해서 휴머나이징
def timeRandomValues():
    time_array = np.linspace(1, 3, 50)
    return np.random.choice(time_array)


search_keyword = "웨딩사진전문"
crowl_count = 1
result = []

# 스크랩 가능한 업체는 총 120곳입니다.(최대)
# 1페이지에 1~100인덱스가 출력되고 2페이지로 넘어가면 11~110이 출력되는 방식입니다.
# 그래서 1페이지 전체 스크랩 + 3페이지의 뒤에서부터 20인덱스까지 뽑아야 중복을 피합니다.

# 가게 고유번호 수집
for page in [1, 3]:

    url = f"https://map.naver.com/v5/api/search?caller=pcweb&query={search_keyword}&type=all&searchCoord=126.75292692474181;37.539870164829594&page={page}&displayCount=100&isPlaceRecommendationReplace=true&lang=ko"
    res = requests.get(url)
    map_data = json.loads(res.text)
    
    # 고유번호 수집
    if page == 3:
        shop_id_list = [info['id'] for info in map_data['result']['place']['list'][-20:]]
    else:
        shop_id_list = [info['id'] for info in map_data['result']['place']['list']]
        

    for shop_id in shop_id_list:

        url = f"https://map.naver.com/v5/api/sites/summary/{shop_id}?lang=ko"
        res = requests.get(url)
        shop_json = json.loads(res.text)

        name = shop_json['name']
        blogreview_count = shop_json['reviewCount']
        phone = shop_json['phone']
        address = shop_json['fullRoadAddress']
        if len(shop_json['urlList']) != 0:
            homepage = shop_json['urlList'][0]['url']
        else:
            homepage = ""
        img_count = len(shop_json['images'])

        result.append([name, blogreview_count, phone, address, homepage, img_count])
        
    
        if img_count == 0:
            continue

        else:
            # 사진 저장 폴더 만들기
            img_dir = f"./img/{name}"
            os.makedirs(img_dir)

            for i, img in enumerate(shop_json['images']):
                urllib.request.urlretrieve(img['url'], img_dir + f"/{img['modDate']}_{i}.jpg")
        
        print(f"{crowl_count}번째 업체 완료")
        crowl_count += 1
        time.sleep(timeRandomValues())

columns = ['이름', '블로그 리뷰수', '연락처', '주소', '홈페이지', '등록사진 수']
df = pd.DataFrame(result, columns=columns)
df.to_csv("result.csv", encoding="utf-8-sig", index=False)

In [ ]:
df